# DADOS DE CARGA DE ENERGIA DIÁRIA

**Descrição do Dado:** Dados de carga por subsistema numa data de referência em base diária.

| Descrição | Código | Tipo de Dado | Formato | Permite valor nulo | Permite valor zerado | Permite valor negativo |
| --- | --- | --- | --- | --- | --- | --- |
| Código do Subsistema | `id_subsistema` | STRING | 3 POSIÇÕES | Não | - | - |
| Nome do Subsistema | `nom_subsistema` | STRING | 60 POSIÇÕES | Não | - | - |
| Data de referência | `din_instante` | DATETIME | `YYY-MM-DD HH:MM:SS` | Não | - | - |
| Valor da Carga de Energia (MWmed) | `val_cargaenergiamwmed` | FLOAT | - | Não | Não | Não |

<small>Última edição em `01-05-2023`.</small>